In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hepatitis"
cohort = "GSE45032"

# Input paths
in_trait_dir = "../../input/GEO/Hepatitis"
in_cohort_dir = "../../input/GEO/Hepatitis/GSE45032"

# Output paths
out_data_file = "../../output/preprocess/Hepatitis/GSE45032.csv"
out_gene_data_file = "../../output/preprocess/Hepatitis/gene_data/GSE45032.csv"
out_clinical_data_file = "../../output/preprocess/Hepatitis/clinical_data/GSE45032.csv"
json_path = "../../output/preprocess/Hepatitis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression in liver of HCC and CHC patients"
!Series_summary	"In order to compare age depenpdence of mRNA between HCC and CHC patients, we measured gene expression by microarray."
!Series_overall_design	"24 liver samples are taken from HCC and CHC patients with various ages and gender."
Sample Characteristics Dictionary:
{0: ['cell type: hepatocallular carcinoma', 'cell type: chronic hepatitis type C'], 1: ['tissue: liver'], 2: ['gender: male', 'gender: female'], 3: ['age(yrs): 67', 'age(yrs): 56', 'age(yrs): 76', 'age(yrs): 79', 'age(yrs): 66', 'age(yrs): 70', 'age(yrs): 68', 'age(yrs): 72', 'age(yrs): 62', 'age(yrs): 55', 'age(yrs): 71', 'age(yrs): 73', 'age(yrs): 74', 'age(yrs): 61', 'age(yrs): 54', 'age(yrs): 64', 'age(yrs): 59', 'age(yrs): 69', 'age(yrs): 25', 'age(yrs): 41', 'age(yrs): 50', 'age(yrs): 58', 'age(yrs): 49', 'age(yrs): 63', 'age(yrs): 60', 'age(yrs): 52', 'age(yrs): 51']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analysis of dataset
# 1. Gene Expression Data Availability
# From the background information, we can see this is a microarray measurement
# of gene expression, so it's likely to contain gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For the trait (Hepatitis), looking at key 0, we see "hepatocallular carcinoma" and "chronic hepatitis type C"
# These are different types of hepatitis conditions, so trait data is available
trait_row = 0

# Age information is available at key 3
age_row = 3

# Gender information is available at key 2
gender_row = 2

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait data to binary (0 for CHC, 1 for HCC)"""
    if pd.isna(value):
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "hepatocallular carcinoma" in value.lower() or "hcc" in value.lower():
        return 1  # HCC
    elif "chronic hepatitis" in value.lower() or "chc" in value.lower():
        return 0  # CHC
    else:
        return None

def convert_age(value):
    """Convert age data to continuous values"""
    if pd.isna(value):
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Extract numeric age value
    try:
        # Remove 'yrs' or other text and convert to float
        age_value = ''.join(c for c in value if c.isdigit() or c == '.')
        return float(age_value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender data to binary (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "female" in value.lower():
        return 0
    elif "male" in value.lower():
        return 1
    else:
        return None

# 3. Save Metadata
# Checking if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a sample characteristics dictionary as provided in the output
    sample_chars_dict = {
        0: ['cell type: hepatocallular carcinoma', 'cell type: chronic hepatitis type C'], 
        1: ['tissue: liver'], 
        2: ['gender: male', 'gender: female'], 
        3: ['age(yrs): 67', 'age(yrs): 56', 'age(yrs): 76', 'age(yrs): 79', 'age(yrs): 66', 
            'age(yrs): 70', 'age(yrs): 68', 'age(yrs): 72', 'age(yrs): 62', 'age(yrs): 55', 
            'age(yrs): 71', 'age(yrs): 73', 'age(yrs): 74', 'age(yrs): 61', 'age(yrs): 54', 
            'age(yrs): 64', 'age(yrs): 59', 'age(yrs): 69', 'age(yrs): 25', 'age(yrs): 41', 
            'age(yrs): 50', 'age(yrs): 58', 'age(yrs): 49', 'age(yrs): 63', 'age(yrs): 60', 
            'age(yrs): 52', 'age(yrs): 51']
    }
    
    # Create a DataFrame with sample IDs as columns and characteristics as rows
    # This matches the expected format for geo_select_clinical_features
    sample_ids = [f"Sample_{i+1}" for i in range(max(len(values) for values in sample_chars_dict.values()))]
    clinical_data = pd.DataFrame(index=range(max(sample_chars_dict.keys()) + 1), columns=sample_ids)
    
    # Populate the DataFrame with the available sample characteristics
    for row_idx, values in sample_chars_dict.items():
        for col_idx, value in enumerate(values):
            if col_idx < len(sample_ids):
                clinical_data.iloc[row_idx, col_idx] = value
    
    # Extract clinical features
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical)
    print("Preview of clinical data:", preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data: {'Sample_1': [1.0, 67.0, 1.0], 'Sample_2': [0.0, 56.0, 0.0], 'Sample_3': [nan, 76.0, nan], 'Sample_4': [nan, 79.0, nan], 'Sample_5': [nan, 66.0, nan], 'Sample_6': [nan, 70.0, nan], 'Sample_7': [nan, 68.0, nan], 'Sample_8': [nan, 72.0, nan], 'Sample_9': [nan, 62.0, nan], 'Sample_10': [nan, 55.0, nan], 'Sample_11': [nan, 71.0, nan], 'Sample_12': [nan, 73.0, nan], 'Sample_13': [nan, 74.0, nan], 'Sample_14': [nan, 61.0, nan], 'Sample_15': [nan, 54.0, nan], 'Sample_16': [nan, 64.0, nan], 'Sample_17': [nan, 59.0, nan], 'Sample_18': [nan, 69.0, nan], 'Sample_19': [nan, 25.0, nan], 'Sample_20': [nan, 41.0, nan], 'Sample_21': [nan, 50.0, nan], 'Sample_22': [nan, 58.0, nan], 'Sample_23': [nan, 49.0, nan], 'Sample_24': [nan, 63.0, nan], 'Sample_25': [nan, 60.0, nan], 'Sample_26': [nan, 52.0, nan], 'Sample_27': [nan, 51.0, nan]}
Clinical data saved to ../../output/preprocess/Hepatitis/clinical_data/GSE45032.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 62976 rows
First 20 gene IDs:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene IDs observed, these appear to be numeric identifiers (1, 2, 3, etc.)
# rather than standard human gene symbols (which would look like BRCA1, TP53, IL6, etc.)
# Therefore, gene mapping will be required to convert these numeric IDs to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE45032
Line 6: !Series_title = Gene expression in liver of HCC and CHC patients
Line 7: !Series_geo_accession = GSE45032
Line 8: !Series_status = Public on Dec 21 2023
Line 9: !Series_submission_date = Mar 12 2013
Line 10: !Series_last_update_date = Dec 21 2023
Line 11: !Series_summary = In order to compare age depenpdence of mRNA between HCC and CHC patients, we measured gene expression by microarray.
Line 12: !Series_overall_design = 24 liver samples are taken from HCC and CHC patients with various ages and gender.
Line 13: !Series_type = Expression profiling by array
Line 14: !Series_contributor = Y-h,,Taguchi
Line 15: !Series_contributor = Yoshiki,,Murakami
Line 16: !Series_sample_id = GSM1096016
Line


Gene annotation preview:
{'ID': [1, 2, 3, 4, 5], 'ProbeName': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P326296', 'A_24_P287941'], 'GB_ACC': [nan, nan, nan, 'NM_144987', 'NM_013290'], 'ControlType': [1, 1, 1, 0, 0], 'accessions': [nan, nan, nan, 'ref|NM_144987|ref|NM_001040425|ens|ENST00000292879|ens|ENST00000392196', 'ref|NM_013290|ref|NM_016556|ens|ENST00000393795|ens|ENST00000253789'], 'GeneName': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'U2AF1L4', 'PSMC3IP'], 'Description': [nan, nan, nan, 'ref|Homo sapiens U2 small nuclear RNA auxiliary factor 1-like 4 (U2AF1L4), transcript variant 2, mRNA [NM_144987]', 'ref|Homo sapiens PSMC3 interacting protein (PSMC3IP), transcript variant 1, mRNA [NM_013290]'], 'chr_coord': [nan, nan, nan, 'hs|chr19:036235296-036235237', 'hs|chr17:040724775-040724716'], 'SEQUENCE': [nan, nan, nan, 'GTATGGGGAGATTGAAGAGATGAATGTGTGCGACAACCTTGGGGACCACGTCGTGGGCAA', 'AAATTGCAGTAGCTTGAGGTTAACATTTAGACTTGGAACAATGCTAAAGGAAAGCATTTG'], 'SPOT_ID': ['--G

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene expression data and gene annotation preview, 
# we need to identify the correct columns for mapping
# From the gene expression data, the gene identifiers are numeric (1, 2, 3...)
# From the gene annotation, 'ID' column matches these numeric identifiers, 
# and 'GeneName' contains the gene symbols we need

# 2. Create gene mapping dataframe from annotation data
prob_col = 'ID'  # Column containing probe IDs matching gene expression data
gene_col = 'GeneName'  # Column containing gene symbols

# Extract the mapping between ID and GeneName
try:
    print("Creating gene ID to symbol mapping...")
    gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
    print(f"Created mapping with {len(gene_mapping)} entries")
    print("First 5 entries of gene mapping:")
    print(gene_mapping.head())
except Exception as e:
    print(f"Error creating gene mapping: {e}")
    raise

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
try:
    print("\nApplying gene mapping to convert probe measurements to gene expression...")
    gene_data = apply_gene_mapping(gene_data, gene_mapping)
    print(f"Converted gene expression data with {len(gene_data.index)} unique genes")
    print("First 10 gene symbols:")
    print(gene_data.index[:10])
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error applying gene mapping: {e}")
    raise


Creating gene ID to symbol mapping...
Created mapping with 62976 entries
First 5 entries of gene mapping:
  ID             Gene
0  1  GE_BrightCorner
1  2       DarkCorner
2  3       DarkCorner
3  4          U2AF1L4
4  5          PSMC3IP

Applying gene mapping to convert probe measurements to gene expression...


Converted gene expression data with 20147 unique genes
First 10 gene symbols:
Index(['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AA081107', 'AA213559'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Hepatitis/gene_data/GSE45032.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")

Gene data shape before normalization: (20147, 48)
Gene data shape after normalization: (19274, 48)


Normalized gene data saved to ../../output/preprocess/Hepatitis/gene_data/GSE45032.csv
Clinical data saved to ../../output/preprocess/Hepatitis/clinical_data/GSE45032.csv
Linked data shape: (75, 19277)

Handling missing values...
After missing value handling, linked data shape: (0, 2)
Skipping bias evaluation due to insufficient data.
Abnormality detected in the cohort: GSE45032. Preprocessing failed.

Dataset usability: False
Dataset is not usable for Hepatitis association studies. Data not saved.
